# reproduce the yang zhang paper (1104.2487v2) on weighting for magnification bias

We need a lot of averages for this, all with respect to magnitude, in particular we need:

$\alpha-1$
$b_g$ and
$W$

$W$ itself requires the same averages.

In [1]:
# from magmod import shotnoise, C_l_DM_CAMB, bgal_new, sg
from magmod import *
from magbias_experiments import SKA_zhangpen, CLAR_zhangpen, SKA1, SKA2, cb_hirax as hirax, hirax512, LSST, LSST_nosgfit, n

MAXMAG = 27


#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$ 
Rescaling the galaxy number density by a factor of 104185.986576 to match the gold sample with 6330073646.61 total galaxies 
#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$ 



bgal_new has the same magnitude dependence as sg, ONLY ON THE BIN CENTER

shotnoise for the magnitude bin is straight forward

C_DM does not depend on magnitude

In [2]:
def Ngal_in_bin(zmin, mtab_edges, galsurv, NINT = 2000, maxmag = MAXMAG, ZMAX = False):
    """mtab_edges is magnitude bin edges, 
    upper means higher number and fainter, thus more galaxies"""

    if type(ZMAX)==bool and not ZMAX:
        zmax = zbg_max(maxmag, galsurv)
    elif type(ZMAX)==bool and ZMAX:
        raise ValueError("ZMAX must either be False or float!")
    else:
        zmax = ZMAX
    z_integrate = np.linspace(zmin,zmax, NINT)


    dNdztab = np.array([nofz(z_integrate, mmm) for mmm in mtab_edges])
    dNdzdOm = np.trapz(dNdztab, z_integrate, axis = 1)
    

    
    #now subtract lower from upper
    Nz = (dNdzdOm[1:] - dNdzdOm[:-1]) * 4 * np.pi #already in rad!!!! and multiplying with all sky
    Nz = np.atleast_1d(Nz)
    if (Nz<0).any():
#         print Nz
        raise ValueError("cannot have negative number of galaxies, most likely we confused magnitudes here")
    return Nz





def alphaminusone(z, maxmag, exp = LSST): #2(alpha-1) = 5sg-2
    """should take the mean redshift in the bin!"""
    maxmag = np.atleast_1d(maxmag)
    res = np.array( [ 5*sg(z, exp, mmm) - 2 for mmm in maxmag])
    return res

def average_no_W(A,B,N, VECTORINPUT = False):
    """A,B are the arrays to be averaged, same length as N which is the bg number density of galaxies"""
    if VECTORINPUT:
        sumaxis = 0
    else:
        sumaxis = 1
    return np.sum(A*B*N, axis = sumaxis)/np.sum(N) #A and B are matrices, ell x mag



def W_weight(C_DMtab, Cshottab, alpha_m_one_tab, bgal_tab, Ngal_tab, exp = LSST):
    """the optimal, scale dependent weight, eq. 10
    input: background redshift z, magnitude bin center mag (later will be full array, now just needs to be inside the table used for bgal_tab),
    array (for ell values) of dark matter power spectrum and the shot noise (number). 
    Then there are tables of alpha-1, bgal, Cshot and Ngal on a given magnitude binning (the binning itself not needed here)"""

    DM_by_shot = np.outer(C_DMtab,1/Cshottab) #this need to have right shape, as C_DM is length ell and Cshot is length magtab
    num = -average_no_W(alpha_m_one_tab, bgal_tab, Ngal_tab, VECTORINPUT = True) * DM_by_shot
    denom = 1 + average_no_W(bgal_tab,bgal_tab, Ngal_tab, VECTORINPUT = True) * DM_by_shot
    fac1 = num/denom

    
#     return np.outer( np.ones( len(C_DMtab)),alpha_m_one_tab) + bgal_tab * fac1 #outer product to get right shape
    return alpha_m_one_tab + bgal_tab * fac1 #outer product to get right shape



# def W_weight_singlebin(z, mag, C_DMtab, Cshottt, alpha_m_one_tab, bgal_tab, Ngal_tab, exp = LSST):
#     """the optimal, scale dependent weight, eq. 10
#     input: mean background redshift z, magnitude bin center mag (later will be full array, now just needs to be inside the table used for bgal_tab),
#     array (for ell values) of dark matter power spectrum and the shot noise (number). 
#     Then there are tables of alpha-1, bgal, Cshot and Ngal on a given magnitude binning (the binning itself not needed here)"""

#     DM_by_shot = C_DMtab/Cshottt #this need to have right shape, as C_DM is length ell and Cshot is length magtab
    
#     num = -average_no_W(alpha_m_one_tab, bgal_tab, Ngal_tab) * DM_by_shot
#     denom = 1 + average_no_W(bgal_tab,bgal_tab, Ngal_tab) * DM_by_shot
#     fac1 = num/denom
#     bgalll = bgal_new(z, mag, experiment = exp)
#     almino = alphaminusone(z, mag, exp = exp)
    
    
#     print bgalll, "bg"
#     print almino, "alpha minus one"
#     print Cshottt, "Cshot"
    
#     return almino + fac1 * bgalll
    
    

def S2N_weighted(ltab, deltaell, fsky, Cfg, CSfg, CSbg, CHImu, biasg, Weight, Ngal):
    num = (2*ltab + 1) * deltaell * fsky
    denom = 1 + (Cfg + CSfg) * ( 
        average_no_W(biasg, Weight, Ngal)**2 * C_DM_tab + 
#         average_no_W(Weight**2, CSbg, Ngal)) / (average_no_W(CHImu, Weight, Ngal)**2) #I put the shot noise into the average
        average_no_W(Weight, Weight, Ngal) * CSbg) / (average_no_W(CHImu, Weight, Ngal)**2)
    frac = num/denom
    res = np.sum(frac, axis = 0)
    return np.sqrt(res)

testing things:

In [21]:
# B1 1st bin:
zflow = 0.34127101; zfhigh = 0.84127101; zfmean = (zfhigh+zflow)/2; delta_zf = (zfhigh-zflow)/2

#background z:
buffer_z = 0.1; 
dzb = 0.1
zlow_LSST = zfhigh + buffer_z

zmax_LSST = LSST["zmax"]

#zhigh should not be needed

#small mag range:
magmin = 25; magmax = 27; 
# Nmag = 2
Nmag = 8
# Nmag = 30
magtab_edges = np.linspace(magmin, magmax, Nmag+1)
print magtab_edges
magtab_centers = (magtab_edges[1:] + magtab_edges[:-1])/2

mag_test = magtab_centers[1]
magmin_test = magtab_edges[1]; magmax_test = magtab_edges[2]


#small ell range
SKAarea = SKA['S_area']
lminSKA = np.amax([10,np.int(np.around(2*pi/np.sqrt(SKAarea)))]) #never use lmin <20
# lminSKA = 200
lminSKA*=5
print "WARNING: lminSKA is set to large value {}!".format(lminSKA)
lend = 2200;
ltabSKA = np.arange(lminSKA, lend + lminSKA, lminSKA)


[25.   25.25 25.5  25.75 26.   26.25 26.5  26.75 27.  ]


We split the background bin into fine bins of dz = 0.1 and add their S2N up in quadrature.

In [22]:
zb_tab, zb_step = np.linspace(zlow_LSST, zmax_LSST, np.int((zmax_LSST-zlow_LSST)/dzb+1), retstep = True)
print zb_tab, zb_step

[0.94127101 0.99141896 1.04156691 1.09171486 1.14186281 1.19201075
 1.2421587  1.29230665 1.3424546  1.39260255 1.4427505  1.49289845
 1.5430464  1.59319435 1.6433423  1.69349024 1.74363819 1.79378614
 1.84393409 1.89408204 1.94422999 1.99437794 2.04452589 2.09467384
 2.14482179 2.19496973 2.24511768 2.29526563 2.34541358 2.39556153
 2.44570948 2.49585743 2.54600538 2.59615333 2.64630128 2.69644922
 2.74659717 2.79674512 2.84689307 2.89704102 2.94718897 2.99733692
 3.04748487 3.09763282 3.14778077 3.19792871 3.24807666 3.29822461
 3.34837256 3.39852051 3.44866846 3.49881641 3.54896436 3.59911231
 3.64926026 3.6994082  3.74955615 3.7997041  3.84985205 3.9       ] 0.05014794898305085


In [23]:
alphaminusones = alphaminusone(zb_tab+zb_step/2, magtab_centers) #zb_step added to get mean redshift. shape (mag X ell)

In [24]:
S2Ntab = []

NINTEGRATE = 5 #coarse because z bins are small
for iz in range(len(zb_tab)-1):#all but the last entry

    zlow = zb_tab[iz]
    zhigh = zb_tab[iz+1]
    zbmean = (zlow + zhigh)/2

    alphaminusone_tab = alphaminusones[:,iz]

    C_DM_tab = C_l_DM_CAMB(ltabSKA, zlow, galsurv = LSST, ZMAX = zhigh)

    Cshot = shotnoise(zlow, LSST, MAXMAG, ZMAX = zhigh, NINT = NINTEGRATE) #coarse integration okay because we use thin redshift bin 

    print zbmean, magtab_centers
    bias_g_tab = np.array([bgal_new(zbmean, mmm) for mmm in magtab_centers])
    N_g_tab = Ngal_in_bin(zlow,magtab_edges, LSST, ZMAX = zhigh, NINT = NINTEGRATE) #takes the lower redshift edge


    Weight = W_weight(C_DM_tab, Cshot, alphaminusone_tab, bias_g_tab, N_g_tab)

    C_HIHI_tab = C_l_HIHI_CAMB(ltabSKA, zflow, zfhigh)
    C_noise_fg_tab = noise_cls_single_dish(ltabSKA, ztonu21(zfmean), SKA1, 256) * np.ones( len(ltabSKA) )
    C_HIxmag_tab = np.array( [Cl_HIxmag_CAMB(ltabSKA, zfmean, delta_zf, zlow, MAXMAG = mmm, ZMAX = zhigh, NINT_gkernel = NINTEGRATE)
                              for mmm in magtab_centers]).T #transpose to match shape

    print "delta ell = {}".format(lminSKA)
    fsky = SKA1["S_area"] / (4*np.pi)

    S2N = S2N_weighted(ltabSKA, lminSKA, fsky, C_HIHI_tab, C_noise_fg_tab, Cshot, C_HIxmag_tab, bias_g_tab, Weight, N_g_tab)
    S2Ntab.append(S2N)
    if np.isnan(S2N):
        break #no more S2N to get

0.9663449844915255 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.0164929334745763 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.066640882457627 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.116788831440678 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.166936780423729 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.2170847294067797 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.2672326783898304 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.3173806273728814 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.3675285763559322 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.4176765253389831 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
1.467824474322034 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta e

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: overflow encountered in true_divide


2.9722629438135595 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
3.02241089279661 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
3.072558841779661 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]
delta ell = 50
3.1227067907627117 [25.125 25.375 25.625 25.875 26.125 26.375 26.625 26.875]


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: overflow encountered in multiply
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in true_divide


delta ell = 50


In [25]:
S2Ntab = np.array(S2Ntab)
print S2Ntab
print np.sqrt(np.nansum(S2Ntab**2))

[ 6.13369731  7.26570455  8.40554358  9.53017312 10.62590181 11.6847965
 12.69902806 13.65612705 14.53731958 15.31649665 15.96130497 16.43484986
 16.69846281 16.71540437 16.45543045 15.90011398 15.04742787 13.91532551
 12.54353811 10.99330168  9.26553096  7.51445369  5.85801038  4.28685553
  2.99208299  1.92301393  1.13573413  0.67735247  0.41463191  0.39702451
  0.49395804  0.67302763  0.90941204  1.08582277  1.18875842  1.19372603
  1.08279181  0.92443135  0.72834913  0.52464263  0.36004074  0.23363477
  0.14232007         nan]
61.79236181849793


This is a massive improvement of S2N when compared to the optimization approach, about a factor of 10!


-- we must keep in mind that we fixed (alpha - 1)bg ourselves, and this has a big impact here too.